> Note: The visualization of this notebook depends on usage of SAPIEN's `Viewer`, which is not available on Colab. **You need to run this notebook locally**. Besides this notebook, you can also find the latest SAPIEN tutorial at [SAPIEN's documentation](https://sapien.ucsd.edu/docs/latest/index.html).

# Motion Planning Tutorial 2: Collision Avoidance

In *Motion Planning Tutorial 1: Plan a Path*, we talked about how to plan paths for the robot. However, in that tutorial, we didn’t take the environment model into account. The robot will avoid self-collisions (i.e., collisions between the robot links), but may collide with the environment. In this tutorial, you will learn two ways to avoid collisions:

- Add environment point clouds to avoid collision
- Attach boxes to avoid collision

<a class="reference internal image-reference" href="../assets/collision1.gif"><img alt="../assets/collision1.gif" src="../assets/collision1.gif" style="width: 32%;" /></a>
<a class="reference internal image-reference" href="../assets/collision2.gif"><img alt="../assets/collision2.gif" src="../assets/collision2.gif" style="width: 32%;" /></a>
<a class="reference internal image-reference" href="../assets/collision3.gif"><img alt="../assets/collision3.gif" src="../assets/collision3.gif" style="width: 32%;" /></a>

The above figures show the benefit of collision avoidance:

- Left: w/o point cloud, w/o attach. The robot arm hits the blue box.
- Middle: w/ point cloud, w/o attach. The red box hits the blue box.
- Right: w/ point cloud, w/ attach. There is no collision.

## Preparation

In [ ]:
%pip install sapien
%pip install mplib
%pip install trimesh

import sapien.core as sapien
import mplib
import numpy as np
import trimesh
from sapien.utils import Viewer

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    raise RuntimeError("The visualization of this notebook cannot be run on Colab! You need to run it locally with a GPU.")

We will be using similar code in *Motion Planning Tutorial 2: Collision Avoidance*:

In [ ]:
# Initialize engine and renderer
engine = sapien.Engine()
engine.set_log_level('critical')
renderer = sapien.SapienRenderer()
engine.set_renderer(renderer)

# Build scene
scene_config = sapien.SceneConfig()
scene = engine.create_scene(scene_config)
scene.set_timestep(1 / 240.0)
scene.add_ground(-0.8)
physical_material = scene.create_physical_material(1, 1, 0.0)
scene.default_physical_material = physical_material

builder = scene.create_actor_builder()
builder.add_box_collision(half_size=[0.4, 0.4, 0.025])
builder.add_box_visual(half_size=[0.4, 0.4, 0.025])
table = builder.build_kinematic(name='table')
table.set_pose(sapien.Pose([0.56, 0, - 0.025]))

# boxes
builder = scene.create_actor_builder()
builder.add_box_collision(half_size=[0.02, 0.02, 0.06])
builder.add_box_visual(half_size=[0.02, 0.02, 0.06], color=[1, 0, 0])
red_cube = builder.build(name='red_cube')
red_cube.set_pose(sapien.Pose([0.7, 0, 0.06]))

builder = scene.create_actor_builder()
builder.add_box_collision(half_size=[0.04, 0.04, 0.005])
builder.add_box_visual(half_size=[0.04, 0.04, 0.005], color=[0, 1, 0])
green_cube = builder.build(name='green_cube')
green_cube.set_pose(sapien.Pose([0.4, 0.3, 0.005]))

builder = scene.create_actor_builder()
builder.add_box_collision(half_size=[0.05, 0.2, 0.1])
builder.add_box_visual(half_size=[0.05, 0.2, 0.1], color=[0, 0, 1])
blue_cube = builder.build(name='blue_cube')
blue_cube.set_pose(sapien.Pose([0.55, 0, 0.1]))

# Set light for visualization
scene.set_ambient_light([0.5, 0.5, 0.5])
scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5], shadow=True)
scene.add_point_light([1, 2, 2], [1, 1, 1], shadow=True)
scene.add_point_light([1, -2, 2], [1, 1, 1], shadow=True)
scene.add_point_light([-1, 0, 1], [1, 1, 1], shadow=True)

# Load robot
loader = scene.create_urdf_loader()
robot = loader.load("../assets/panda/panda.urdf")
robot.set_root_pose(sapien.Pose([0, 0, 0], [1, 0, 0, 0]))

# Set initial joint positions
init_qpos = [0, 0.19634954084936207, 0.0, -2.617993877991494, 0.0, 2.941592653589793, 0.7853981633974483, 0, 0]
robot.set_qpos(init_qpos)

# Set up planner
link_names = [link.get_name() for link in robot.get_links()]
joint_names = [joint.get_name() for joint in robot.get_active_joints()]
planner = mplib.Planner(
    urdf="../assets/panda/panda.urdf",
    srdf="../assets/panda/panda.srdf",
    user_link_names=link_names,
    user_joint_names=joint_names,
    move_group="panda_hand",
    joint_vel_limits=np.ones(7),
    joint_acc_limits=np.ones(7)
)

# Controller
active_joints = robot.get_active_joints()
for joint in active_joints:
    joint.set_drive_property(stiffness=1000, damping=200)

def follow_path(result, viewer): # parameter viewer is for later visualization
    n_step = result['position'].shape[0]
    for i in range(n_step):  
        qf = robot.compute_passive_force(
            gravity=True, 
            coriolis_and_centrifugal=True)
        robot.set_qf(qf)
        for j in range(7):
            active_joints[j].set_drive_target(result['position'][i][j])
            active_joints[j].set_drive_velocity_target(result['velocity'][i][j])
        scene.step()
        if i % 4 == 0:
            scene.update_render()
            viewer.render()

def open_gripper(viewer): # parameter viewer is for later visualization
    for joint in active_joints[-2:]:
        joint.set_drive_target(0.4)
    for i in range(100): 
        qf = robot.compute_passive_force(
            gravity=True, 
            coriolis_and_centrifugal=True)
        robot.set_qf(qf)
        scene.step()
        if i % 4 == 0:
            scene.update_render()
            viewer.render()

def close_gripper(viewer): # parameter viewer is for later visualization
    for joint in active_joints[-2:]:
        joint.set_drive_target(0)
    for i in range(100):  
        qf = robot.compute_passive_force(
            gravity=True, 
            coriolis_and_centrifugal=True)
        robot.set_qf(qf)
        scene.step()
        if i % 4 == 0:
            scene.update_render()
            viewer.render()

## Add enviroment point clouds

One way to model the environment and avoid collision is through point clouds. The point cloud may come from the sensor observations or be sampled from the mesh surfaces. For example, we can add a point cloud for the blue box with `planner.update_point_cloud()`:

In [ ]:
def add_point_cloud():
    box = trimesh.creation.box([0.1, 0.4, 0.2])
    points, _ = trimesh.sample.sample_surface(box, 1000)
    points += [0.55, 0, 0.1]
    planner.update_point_cloud(points)
    return

`planner.update_point_cloud()` takes two arguments. The first one is a NumPy array of shape $n \times 3$, which describes the coordinates of the points. **The coordinates should be represented in the frame of the robot arm’s root link**. The second (optional) argument is `resolution = 1e-3`, which describes the resolution of each point.

After adding the point cloud, we can avoid collisions between the robot and the point cloud by setting `use_point_cloud` to be True. Both `planner.plan()` and `planner.plan_screw()` support this flag:

In [ ]:
def move_to_pose(pose, use_point_cloud, use_attach, viewer):
    result = planner.plan_screw(pose, robot.get_qpos(), time_step=1/250,
                                    use_point_cloud=use_point_cloud, use_attach=use_attach)
    if result['status'] != "Success":
        result = planner.plan(pose, robot.get_qpos(), time_step=1/250,
                                    use_point_cloud=use_point_cloud, use_attach=use_attach)
        if result['status'] != "Success":
            print(result['status'])
            return -1 
    follow_path(result, viewer)
    return 0

You don’t need to provide the point cloud for each `planner.plan()` or `planner.plan_screw()` call. You can use `planner.update_point_cloud()` to update the point cloud once it’s changed.

> Note: Please remember to remove the points of the robot arm if the points come from the sensor observation. Otherwise, there will always be collisions, and the planner may fail to plan a valid path.

## Attach a box

As shown in the above figure (middle one), after adding the point cloud of the blue box, the robot will not collide with it. However, the red box moves with the robot, and it may still collide with the blue box. To address this issue, we can attach a box to the robot, so that we can avoid the collision between the attached box and the environment point cloud:

In [ ]:
def attach_box(planner):
    planner.update_attached_box([0.04, 0.04, 0.12], [0, 0, 0.14, 1, 0, 0, 0])

`planner.update_attached_box()` takes three arguments:

- `size`: a list with three elements indicates the size of the attached box.
- `pose`: a list with seven elements indicates the relative pose from the box to the attached link. The first three elements describe the position part, and the remaining four elements describe the quaternion (wxyz) for the rotation part.
- `link_id` = -1: optional, an integer indicates the id of the link that the box is attached to. The link id is determined by the `user_link_names` (during Configuration), and starts from 0. The default value -1 indicates the `move_group` link.

After adding the attached box, we can avoid collisions between the attached box and the point cloud by setting both `use_point_cloud` and `use_attach` to be True. Both `planner.plan()` and `planner.plan_screw()` support the flags.

You can use `planner.update_attached_box()` to update the box once it’s changed.

As shown in the above figure (the right one), after adding the point cloud of the blue box and attaching the red box to the `move_group` link, there is no collision.

## Visualization

You can adjust the `use_point_cloud` flag and `use_attach` flag to expore different behaviors of the robot.

In [ ]:
def demo(planner, use_point_cloud, use_attach, viewer):
    pickup_pose = [0.7, 0, 0.12, 0, 1, 0, 0]
    delivery_pose = [0.4, 0.3, 0.13, 0, 1, 0, 0]
    
    if use_point_cloud:
        add_point_cloud()
    
    pickup_pose[2] += 0.2
    move_to_pose(pickup_pose, use_point_cloud, False, viewer) 
    open_gripper(viewer)
    pickup_pose[2] -= 0.12
    move_to_pose(pickup_pose, use_point_cloud, False, viewer)
    close_gripper(viewer)

    if use_attach:
        attach_box(planner)

    pickup_pose[2] += 0.12
    move_to_pose(pickup_pose, use_point_cloud, use_attach, viewer)
    delivery_pose[2] += 0.2
    move_to_pose(delivery_pose, use_point_cloud, use_attach, viewer)
    delivery_pose[2] -= 0.12
    move_to_pose(delivery_pose, use_point_cloud, use_attach, viewer)
    open_gripper(viewer)
    delivery_pose[2] += 0.12
    move_to_pose(delivery_pose, use_point_cloud, use_attach, viewer)

In [ ]:
viewer = Viewer(renderer)
viewer.set_scene(scene)
viewer.set_camera_xyz(x=1.2, y=0.25, z=0.4)
viewer.set_camera_rpy(r=0, p=-0.4, y=2.7)

demo(planner, use_point_cloud=True, use_attach=True, viewer=viewer)

viewer.close()